In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
df = pd.read_csv("/kaggle/input/heart-attack-new/heart_attack_russia_youth_vs_adult.csv")
print(df.columns)

Index(['ID', 'Age', 'Gender', 'Region', 'Blood_Pressure', 'Cholesterol', 'BMI',
       'Heart_Rate', 'Exercise_Level', 'Smoking', 'Alcohol_Consumption',
       'Diabetes', 'Family_History', 'Stress_Level', 'Heart_Attack', 'Angina',
       'Heart_Disease_History', 'Diet', 'Sleep_Hours', 'Occupation',
       'Income_Level', 'Physical_Activity', 'Education_Level',
       'Marital_Status', 'Urban_Rural', 'Medication', 'Health_Awareness',
       'Daily_Water_Intake', 'Mental_Health', 'Obesity'],
      dtype='object')


In [2]:
df["Heart_Attack"].unique()


array([False,  True])

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# -------------------------------
# 1. Load dataset
# -------------------------------
df = pd.read_csv("/kaggle/input/heart-attack-new/heart_attack_russia_youth_vs_adult.csv")
print("Initial shape:", df.shape)
print("Columns:", df.columns.tolist())

# -------------------------------
# 2. Convert target column True/False → 1/0
# -------------------------------
if df["Heart_Attack"].dtype == bool or df["Heart_Attack"].dtype == object:
    df["Heart_Attack"] = df["Heart_Attack"].map({True: 1, False: 0}).astype(int)

print("\nUnique values in Heart_Attack:", df["Heart_Attack"].unique())

# -------------------------------
# 3. Drop useless columns (if needed)
# -------------------------------
if "ID" in df.columns:
    df = df.drop("ID", axis=1)

# -------------------------------
# 4. Detect numeric and categorical columns
# -------------------------------
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object", "bool"]).columns.tolist()

# Remove target from lists
if "Heart_Attack" in num_cols:
    num_cols.remove("Heart_Attack")

print("\nNumeric columns:", num_cols)
print("Categorical columns:", cat_cols)

# -------------------------------
# 5. Handle missing values
# -------------------------------
# Numeric fill with median
if len(num_cols) > 0:
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical fill with mode
for col in cat_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])

# -------------------------------
# 6. Encode categorical columns
# -------------------------------
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# -------------------------------
# 7. Scale numeric columns
# -------------------------------
scaler = MinMaxScaler()
if len(num_cols) > 0:
    df[num_cols] = scaler.fit_transform(df[num_cols])

# -------------------------------
# 8. Final output
# -------------------------------
output_path = "/kaggle/working/heart_attack_cleaned.csv"
df.to_csv(output_path, index=False)

print("\n✅ Preprocessing Complete!")
print("📁 Saved as:", output_path)
print("Final Shape:", df.shape)
print("Target distribution:")
print(df["Heart_Attack"].value_counts())


Initial shape: (50000, 30)
Columns: ['ID', 'Age', 'Gender', 'Region', 'Blood_Pressure', 'Cholesterol', 'BMI', 'Heart_Rate', 'Exercise_Level', 'Smoking', 'Alcohol_Consumption', 'Diabetes', 'Family_History', 'Stress_Level', 'Heart_Attack', 'Angina', 'Heart_Disease_History', 'Diet', 'Sleep_Hours', 'Occupation', 'Income_Level', 'Physical_Activity', 'Education_Level', 'Marital_Status', 'Urban_Rural', 'Medication', 'Health_Awareness', 'Daily_Water_Intake', 'Mental_Health', 'Obesity']

Unique values in Heart_Attack: [0 1]

Numeric columns: ['Age', 'Blood_Pressure', 'Cholesterol', 'BMI', 'Heart_Rate', 'Stress_Level', 'Sleep_Hours', 'Health_Awareness', 'Daily_Water_Intake', 'Mental_Health']
Categorical columns: ['Gender', 'Region', 'Exercise_Level', 'Smoking', 'Alcohol_Consumption', 'Diabetes', 'Family_History', 'Angina', 'Heart_Disease_History', 'Diet', 'Occupation', 'Income_Level', 'Physical_Activity', 'Education_Level', 'Marital_Status', 'Urban_Rural', 'Medication', 'Obesity']

✅ Preprocessi

In [9]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# -----------------------------
# 1️⃣ Load your dataset
# -----------------------------
df = pd.read_csv("/kaggle/working/heart_attack_smoted.csv")  # change file if needed

TARGET = "Heart_Attack"   # <-- change if your target column is different

X = df.drop(TARGET, axis=1)
y = df[TARGET].astype(int)

print("Original dimension:", X.shape[1])

# -----------------------------
# 2️⃣ Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# -----------------------------
# 3️⃣ XGBoost Model (500 Iterations)
# -----------------------------
model = XGBClassifier(
    n_estimators=500,          # <-- 500 boosting rounds
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# -----------------------------
# 4️⃣ Predictions
# -----------------------------
y_pred = model.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

# -----------------------------
# 5️⃣ Results
# -----------------------------
print("\n===== XGBoost Results (500 Iterations) =====")
print(f"Test Accuracy : {test_acc:.8f}")
print(f"Precision     : {test_prec:.8f}")
print(f"Recall        : {test_rec:.8f}")
print(f"F1 Score      : {test_f1:.8f}")


Original dimension: 28

===== XGBoost Results (500 Iterations) =====
Test Accuracy : 0.92628060
Precision     : 1.00000000
Recall        : 0.85256120
F1 Score      : 0.92041353


In [8]:
# SMOTE the whole dataset and save balanced CSV
import sys
import subprocess
import pandas as pd
import numpy as np
from collections import Counter

# try to import imblearn.SMOTE; install if missing
try:
    from imblearn.over_sampling import SMOTE
except Exception:
    print("imblearn not found -> installing imbalanced-learn...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "imbalanced-learn"])
    from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder

# ------------- CONFIG -------------
INPUT_PATH = "/kaggle/working/heart_attack_cleaned.csv"   # change if needed
TARGET = "Heart_Attack"
OUTPUT_PATH = "/kaggle/working/heart_attack_smoted.csv"
RANDOM_STATE = 42
# ----------------------------------

# 1) Load
df = pd.read_csv(INPUT_PATH)
print("Loaded:", INPUT_PATH)
print("Initial shape:", df.shape)
if TARGET not in df.columns:
    raise ValueError(f"Target column '{TARGET}' not found in dataframe.")

print("\nOriginal target distribution (counts):")
print(df[TARGET].value_counts())

# 2) Separate X, y
X = df.drop(TARGET, axis=1)
y = df[TARGET].astype(int)

# 3) Ensure all features numeric: label-encode object columns (so SMOTE can run)
obj_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
if len(obj_cols) > 0:
    print("\nLabel-encoding object columns for SMOTE:", obj_cols)
    le = LabelEncoder()
    for col in obj_cols:
        X[col] = le.fit_transform(X[col].astype(str))

# 4) Convert any boolean to int (if present)
bool_cols = X.select_dtypes(include=['bool']).columns.tolist()
for col in bool_cols:
    X[col] = X[col].astype(int)

# 5) Final dtype check
non_numeric = X.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric:
    raise ValueError("Found non-numeric columns after encoding: " + ", ".join(non_numeric))

# 6) Apply SMOTE on full data (this creates synthetic minority samples)
smote = SMOTE(random_state=RANDOM_STATE, n_jobs=-1)
X_res, y_res = smote.fit_resample(X, y)

print("\nAfter SMOTE - distribution (counts):")
print(Counter(y_res))

# 7) Rebuild DataFrame and save
df_res = pd.DataFrame(X_res, columns=X.columns)
df_res[TARGET] = y_res

df_res.to_csv(OUTPUT_PATH, index=False)
print("\n✅ Saved balanced dataset to:", OUTPUT_PATH)
print("Balanced shape:", df_res.shape)

# 8) Quick sanity checks
print("\nPreview (first 3 rows):")
print(df_res.head(3).to_string(index=False))

print("\nTarget distribution (final):")
print(df_res[TARGET].value_counts())


/usr/local/lib/python3.11/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x79768c118e00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^

Loaded: /kaggle/working/heart_attack_cleaned.csv
Initial shape: (50000, 29)

Original target distribution (counts):
Heart_Attack
0    44119
1     5881
Name: count, dtype: int64

After SMOTE - distribution (counts):
Counter({0: 44119, 1: 44119})

✅ Saved balanced dataset to: /kaggle/working/heart_attack_smoted.csv
Balanced shape: (88238, 29)

Preview (first 3 rows):
     Age  Gender  Region  Blood_Pressure  Cholesterol      BMI  Heart_Rate  Exercise_Level  Smoking  Alcohol_Consumption  Diabetes  Family_History  Stress_Level  Angina  Heart_Disease_History  Diet  Sleep_Hours  Occupation  Income_Level  Physical_Activity  Education_Level  Marital_Status  Urban_Rural  Medication  Health_Awareness  Daily_Water_Intake  Mental_Health  Obesity  Heart_Attack
0.791667       1       0        0.389408     0.515449 0.300926       0.400               0        0                    1         0               0      0.777778       0                      0     2         0.90           3             1      

In [7]:
!pip install --upgrade "scikit-learn==1.2.2"
!pip install --upgrade "imbalanced-learn==0.10.1"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 56.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.2.2 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: imbalanced-learn
    Found existing instal

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dhoogla/cicddos2019")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cicddos2019


In [3]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/DNS-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [4]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/LDAP-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [5]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/LDAP-training.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [6]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/MSSQL-training.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [7]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/MSSQL-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [8]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/NTP-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [9]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/NetBIOS-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [10]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/NetBIOS-training.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [11]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/Portmap-training.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [12]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/SNMP-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [13]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/Syn-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [14]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/Syn-training.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [15]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/TFTP-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [16]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/UDP-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [17]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/UDP-training.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [18]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/UDPLag-training.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [19]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet("/kaggle/input/cicddos2019/UDPLag-testing.parquet")

# Show columns
print("Columns in the dataset:\n")
print(df.columns.tolist())

# Check unique values in Label
print("\nUnique values in 'Label' column:")
print(df["Label"].unique())

# Count of each Label
print("\nLabel value counts:")
print(df["Label"].value_counts())
print(df.shape)


Columns in the dataset:

['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE

In [20]:
# Combine selected label slices from multiple CIC-DDoS-2019 parquet files
# and save a single CSV for downstream experiments.
#
# Edit `SELECTIONS` if you want to change counts.

import pandas as pd
import os
from collections import Counter

# ------------------------------
# CONFIG: file -> {label: count, ...}
# ------------------------------
SELECTIONS = {
    "/kaggle/input/cicddos2019/DNS-testing.parquet": {
        "Benign": 3034,
        "DrDoS_DNS": 3661
    },
    "/kaggle/input/cicddos2019/LDAP-testing.parquet": {
        "Benign": 1391,
        "DrDoS_LDAP": 1440
    },
    "/kaggle/input/cicddos2019/LDAP-training.parquet": {
        "Benign": 4585,
        "LDAP": 1884,
        "NetBIOS": 246
    },
    "/kaggle/input/cicddos2019/MSSQL-testing.parquet": {
        "Benign": 1871,
        "DrDoS_MSSQL": 3861
    },
    "/kaggle/input/cicddos2019/MSSQL-training.parquet": {
        "Benign": 2574,
        "MSSQL": 3761,
        "LDAP": 22
    },
    "/kaggle/input/cicddos2019/NTP-testing.parquet": {
        "Benign": 13306,
        "DrDoS_NTP": 3879
    },
    "/kaggle/input/cicddos2019/NetBIOS-testing.parquet": {
        "Benign": 1627,
        "DrDoS_NetBIOS": 598
    },
    "/kaggle/input/cicddos2019/NetBIOS-training.parquet": {
        "Benign": 1233,
        "NetBIOS": 398
    },
    "/kaggle/input/cicddos2019/Portmap-training.parquet": {
        "Benign": 4420,
        "Portmap": 685
    },
    "/kaggle/input/cicddos2019/SNMP-testing.parquet": {
        "Benign": 1301,
        "DrDoS_SNMP": 2717
    },
    "/kaggle/input/cicddos2019/Syn-testing.parquet": {
        "Benign": 374,
        "Syn": 533
    },
    "/kaggle/input/cicddos2019/Syn-training.parquet": {
        "Benign": 14284,
        "Syn": 3661
    },
    "/kaggle/input/cicddos2019/TFTP-testing.parquet": {
        "TFTP": 3861
    },
    "/kaggle/input/cicddos2019/UDP-testing.parquet": {
        "DrDoS_UDP": 3861
    },
    "/kaggle/input/cicddos2019/UDP-training.parquet": {
        "UDP": 3861,
        "MSSQL": 145
    },
    "/kaggle/input/cicddos2019/UDPLag-testing.parquet": {
        "UDP-lag": 3861,
        "WebDDoS": 51
    },
    "/kaggle/input/cicddos2019/UDPLag-training.parquet": {
        "Syn": 3661,
        "UDP": 3298,
        "UDPLag": 55
    }
}

OUTPUT_PATH = "/kaggle/working/cicddos2019_selected_combined.csv"
random_state = 42

# ------------------------------
# Processing
# ------------------------------
selected_parts = []
summary = []

for path, label_counts in SELECTIONS.items():
    if not os.path.exists(path):
        print(f"WARNING: file not found -> {path}  (skipping)")
        continue

    print(f"\nLoading: {path}")
    df = pd.read_parquet(path)
    # Ensure 'Label' column exists
    if "Label" not in df.columns:
        print(f"  ERROR: 'Label' column not found in {path} - skipping this file.")
        continue

    for label, req_count in label_counts.items():
        df_label = df[df["Label"] == label]
        available = len(df_label)
        take = min(req_count, available)
        if take == 0:
            print(f"  - Label '{label}' not found in file (requested {req_count}, available {available})")
            continue

        # sampling: if available >= take, sample without replacement, else take all
        if available >= take:
            sampled = df_label.sample(n=take, random_state=random_state)
        else:
            sampled = df_label.copy()  # take all
            print(f"  - NOTE: requested {req_count} rows but only {available} available for label '{label}' in {os.path.basename(path)}; taking {available}.")

        selected_parts.append(sampled)
        summary.append((os.path.basename(path), label, req_count, available, len(sampled)))
        print(f"  -> Selected {len(sampled):5d} rows for label '{label}' (requested {req_count}, available {available})")

# Combine parts
if len(selected_parts) == 0:
    raise RuntimeError("No rows were selected from any file. Check paths / labels in SELECTIONS.")

combined = pd.concat(selected_parts, ignore_index=True).sample(frac=1, random_state=random_state).reset_index(drop=True)

# Save
combined.to_csv(OUTPUT_PATH, index=False)
print("\n\n✅ Combined CSV saved to:", OUTPUT_PATH)
print("Combined shape:", combined.shape)

# Print summary table
print("\nSelection summary (file, label, requested, available, taken):")
for row in summary:
    print(row)

# Print final Label distribution
print("\nFinal 'Label' value counts in combined dataset:")
print(combined["Label"].value_counts())

# Optional: show columns and sample head
print("\nColumns (count={}):".format(len(combined.columns)))
print(combined.columns.tolist())
print("\nSample rows:")
print(combined.head(3).to_string(index=False))



Loading: /kaggle/input/cicddos2019/DNS-testing.parquet
  -> Selected  3034 rows for label 'Benign' (requested 3034, available 3034)
  -> Selected  3661 rows for label 'DrDoS_DNS' (requested 3661, available 3669)

Loading: /kaggle/input/cicddos2019/LDAP-testing.parquet
  -> Selected  1391 rows for label 'Benign' (requested 1391, available 1391)
  -> Selected  1440 rows for label 'DrDoS_LDAP' (requested 1440, available 1440)

Loading: /kaggle/input/cicddos2019/LDAP-training.parquet
  -> Selected  4585 rows for label 'Benign' (requested 4585, available 4585)
  -> Selected  1884 rows for label 'LDAP' (requested 1884, available 1884)
  -> Selected   246 rows for label 'NetBIOS' (requested 246, available 246)

Loading: /kaggle/input/cicddos2019/MSSQL-testing.parquet
  -> Selected  1871 rows for label 'Benign' (requested 1871, available 1871)
  -> Selected  3861 rows for label 'DrDoS_MSSQL' (requested 3861, available 6212)

Loading: /kaggle/input/cicddos2019/MSSQL-training.parquet
  -> Selec

In [21]:
d=pd.read_csv("/kaggle/working/cicddos2019_selected_combined.csv")
d.shape

(100000, 78)

In [22]:
d.columns

Index(['Protocol', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Fwd Packets Length Total',
       'Bwd Packets Length Total', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Fla

In [23]:
print("Columns:\n", d.columns.tolist())

print("\nUnique values in Label:")
print(d["Label"].unique())

print("\nLabel value counts:")
print(d["Label"].value_counts())


Columns:
 ['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', '

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# -------------------------------
# 1. Load dataset
# -------------------------------
df = pd.read_csv("/kaggle/working/cicddos2019_selected_combined.csv")  # your combined file
print("Initial shape:", df.shape)
print("Columns:", df.columns.tolist())

# -------------------------------
# 2. Convert Label column → binary (Benign = 0, Others = 1)
# -------------------------------
df["Label"] = df["Label"].apply(lambda x: 0 if x == "Benign" else 1).astype(int)

print("\nUnique values in Label:", df["Label"].unique())
print("Label counts:\n", df["Label"].value_counts())

# -------------------------------
# 3. Drop useless columns (if any)
# -------------------------------
if "ID" in df.columns:
    df = df.drop("ID", axis=1)

# -------------------------------
# 4. Detect numeric + categorical columns
# -------------------------------
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object", "bool"]).columns.tolist()

# Remove target from numeric/categorical lists
if "Label" in num_cols:
    num_cols.remove("Label")
if "Label" in cat_cols:
    cat_cols.remove("Label")

print("\nNumeric columns:", num_cols)
print("Categorical columns:", cat_cols)

# -------------------------------
# 5. Handle missing values
# -------------------------------
# Numeric → fill with median
if len(num_cols) > 0:
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical → fill with mode
for col in cat_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])

# -------------------------------
# 6. Label-encode categorical columns
# -------------------------------
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# -------------------------------
# 7. Scale numeric columns
# -------------------------------
scaler = MinMaxScaler()
if len(num_cols) > 0:
    df[num_cols] = scaler.fit_transform(df[num_cols])

# -------------------------------
# 8. Save cleaned dataset
# -------------------------------
output_path = "/kaggle/working/cicddos2019_cleaned_final.csv"
df.to_csv(output_path, index=False)

print("\n✅ Preprocessing Complete!")
print("📁 Saved as:", output_path)
print("Final Shape:", df.shape)

print("\nFinal Label distribution:")
print(df["Label"].value_counts())


Initial shape: (100000, 78)
Columns: ['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag 

In [26]:
df = pd.read_csv("/kaggle/working/cicddos2019_cleaned_final.csv")

X = df.drop("Label",axis=1)
y = df["Label"].astype(int)

In [27]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Load dataset

print("Original dimension:", X.shape[1])

# -----------------------------
# Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# -----------------------------
# XGBoost Model
# -----------------------------
model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# -----------------------------
# Predictions
# -----------------------------
y_pred = model.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

# -----------------------------
# Final Results
# -----------------------------
print("\n===== XGBoost Results =====")
print(f"Test Accuracy : {test_acc:.8f}")
print(f"Precision     : {test_prec:.8f}")
print(f"Recall        : {test_rec:.8f}")
print(f"F1 Score      : {test_f1:.8f}")

Original dimension: 77

===== XGBoost Results =====
Test Accuracy : 0.99885000
Precision     : 0.99939934
Recall        : 0.99830000
F1 Score      : 0.99884937


In [28]:
print("hi")

hi


pca 2019cicid

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from xgboost import XGBClassifier   # 🔥 NEW

df = pd.read_csv("/kaggle/working/cicddos2019_cleaned_final.csv")

X = df.drop("Label",axis=1)
y = df["Label"].astype(int)

# 1) Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2) PCA to retain 95% variance
pca = PCA(n_components=0.95)  
X_pca = pca.fit_transform(X_scaled)

print("Original dimension:", X.shape[1])
print("PCA dimension:", X_pca.shape[1])

# 3) Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.20, stratify=y, random_state=42
)

# ---------------------------------------------------
# 4) ❗ Replace CatBoost with XGBoost
# ---------------------------------------------------
model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist"        # Fast training
)

# Fit
model.fit(X_train, y_train)

# Test prediction
y_pred = model.predict(X_test)

# Metrics
test_acc = accuracy_score(y_test, y_pred)
test_prec = precision_score(y_test, y_pred, zero_division=0)
test_rec = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

print("\n=== PCA + XGBOOST MODEL RESULTS ===")
print("Test Accuracy :", test_acc)
print("Precision     :", test_prec)
print("Recall        :", test_rec)
print("F1 Score      :", test_f1)


Original dimension: 77
PCA dimension: 24

=== PCA + XGBOOST MODEL RESULTS ===
Test Accuracy : 0.9985
Precision     : 0.9987992795677406
Recall        : 0.9982
F1 Score      : 0.9984995498649595


chi square

In [30]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier   # ← changed

# -------------------------
# 1. Prepare data
# -------------------------

print("Original dimension:", X.shape[1])

# Train-test split (same style as your hybrid pipeline)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

# -------------------------
# 2. Scale to non-negative for chi-square
# -------------------------
scaler = MinMaxScaler()   # maps features to [0, 1]
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -------------------------
# 3. Try different numbers of selected features
# -------------------------
k_values = [5, 10, 15, 20, 16,13]

results_chi2 = {}

for k in k_values:
    print("\n" + "="*60)
    print(f"CHI-SQUARE + XGBoost with top-{k} features")
    print("="*60)

    # 3.1 Chi-Square feature selection
    selector = SelectKBest(score_func=chi2, k=k)
    X_train_k = selector.fit_transform(X_train_scaled, y_train)
    X_test_k = selector.transform(X_test_scaled)

    # Get selected feature names (from original X)
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    print(f"Selected {k} features:")
    print(selected_features)

    # 3.2 Train XGBoost on selected features
    model = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        use_label_encoder=False,   # avoid label encoder warning
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_k, y_train)

    # 3.3 Evaluate on test set
    y_pred = model.predict(X_test_k)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    print(f"Test Accuracy : {acc:.6f}")
    print(f"Precision     : {prec:.6f}")
    print(f"Recall        : {rec:.6f}")
    print(f"F1 Score      : {f1:.6f}")

    # store results for later comparison
    results_chi2[k] = {
        "features": selected_features,
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

print("\n=========== SUMMARY: Chi-Square + XGBoost ===========")
for k, info in results_chi2.items():
    print(f"Top-{k} features -> F1={info['f1']:.6f}, Acc={info['acc']:.6f}")


Original dimension: 77

CHI-SQUARE + XGBoost with top-5 features
Selected 5 features:
['Fwd Packet Length Min', 'Packet Length Min', 'URG Flag Count', 'CWE Flag Count', 'Avg Fwd Segment Size']
Test Accuracy : 0.967400
Precision     : 0.945907
Recall        : 0.991500
F1 Score      : 0.968167

CHI-SQUARE + XGBoost with top-10 features
Selected 10 features:
['Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Flow Bytes/s', 'Fwd PSH Flags', 'Packet Length Min', 'RST Flag Count', 'URG Flag Count', 'CWE Flag Count', 'Avg Packet Size', 'Avg Fwd Segment Size']
Test Accuracy : 0.970700
Precision     : 0.947178
Recall        : 0.997000
F1 Score      : 0.971451

CHI-SQUARE + XGBoost with top-15 features
Selected 15 features:
['Protocol', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Flow Bytes/s', 'Flow Packets/s', 'Fwd PSH Flags', 'Fwd Packets/s', 'Packet Length Min', 'Packet Length Mean', 'RST Flag Count', 'URG Flag Count', 'CWE Flag Count', 'Avg Packet Size', 'Avg Fwd Segment Size', 'I

KeyboardInterrupt: 